In [ ]:
import os
import sys

import pandas as pd
import numpy as np
import gensim
import tensorflow as tf

%matplotlib inline
from IPython.display import clear_output
from IPython.core.pylabtools import figsize
from seaborn import plt

pd.options.display.max_columns = 999
pd.options.display.max_rows = 100

In [ ]:
def t_softmax(x, temp):
    exps = tf.exp(x / temp)
    return exps / tf.reduce_sum(exps, axis=1, keep_dims=True)


def t_squaremax(x, temp):
    sq = tf.square((x + temp) / temp)
    return sq / tf.reduce_sum(sq, axis=1, keep_dims=True)


def min_max_scale(x, k=1.0):
    mins = tf.reduce_min(x, axis=1, keep_dims=True)
    scales = tf.reduce_max(x, axis=1, keep_dims=True) - mins
    return ((x - mins) / scales) * k


def leaky_relu(leak=0.03):
    def lrelu(x):
        pos = tf.cast(x >= 0, tf.float32)
        return (pos * x) + ((1 - pos) * x * leak)
    return lrelu


class ReuseableConv1D(object):
    
    def __init__(self, in_ch, out_ch, act=None):
        self._act = act
        with tf.variable_scope("ReuseableConv1D"):
            w_init = tf.random_normal([1, in_ch, out_ch], mean=0, stddev=1.0 / in_ch)
            self.w = tf.Variable(w_init, name="Weight")
            self.b = tf.Variable(tf.zeros([out_ch]), name="Bias")
    
    def __call__(self, x):
        with tf.variable_scope("ReuseableConv1D"):
            h = tf.nn.conv1d(x, self.w, 1, "VALID") + self.b
            if self._act is None:
                return h
            else:
                return self._act(h)


def reuseable_convs(pattern, name, final=None):
    with tf.variable_scope(name):
        return [
            ReuseableConv1D(
                pattern[i], pattern[i + 1],
                act=final if i == len(pattern) - 2 else leaky_relu())
            for i in np.arange(len(pattern) - 1)]
    

class BasketGenerator(object):
    
    def __init__(
            self, sess=None, name=None, n_choices=3, n_candidates=50,
            softmax_fn=t_squaremax, candidate_ch=32, context_ch=32,
            selector_hidden_ch=[32, 16], context_hidden_ch=[32, 32],
            context_encoder_ch=[32, 32, 32], verbose=1):

        # Parameters
        self._n_choices = n_choices
        self._n_cnd = n_candidates
        self._softmax = softmax_fn
        self._cnd_ch = candidate_ch
        self._cxt_ch = context_ch
        self._cxt_enc_ch_pattern = context_encoder_ch
        self._cxt_enc_ch = context_encoder_ch[-1]
        self._total_ch = candidate_ch + self._cxt_enc_ch
        self._sel_ch_pattern = [self._total_ch] + selector_hidden_ch + [1]
        self._cxt_ch_pattern = [self._total_ch] + context_hidden_ch + [self._cxt_enc_ch]
        self.verbose = 1
        
        # Netork Initialization
        self._sess = sess or tf.get_default_session() or tf.InteractiveSession()
        self._name = "BasketNet_{}".format(name or np.random.randint(0, 1000000))
        self._build()
        self._sess.run(tf.global_variables_initializer())
        
        # Book keeping
        self._epochs = 0
        self._batches = 0
        self._examples = [0]
        self._lrs = []
        self._train_losses = []
        self._temperatures = []
        self._learn_rates = []
        self._valid_examples = []
        self._valid_losses = []
    
    def fit(
            self, train_cnd_cxt_targets, val_cnd_cxt_targets,
            batch_size=64, epochs=1, temp_halflife=1000 * 1000, lr=0.002,
            report_every=100, validate_at_start=False):
        if validate_at_start:
            self._validate(
                *val_cnd_cxt_targets,
                batch_size=batch_size, report_every=report_every)
        for ep in np.arange(epochs):
            ep_lr = lr[ep] if isinstance(lr, (list, tuple)) else lr
            self._train_epoch(
                *train_cnd_cxt_targets,
                batch_size=batch_size, temp_halflife=temp_halflife,
                lr=ep_lr, report_every=report_every)
            self._validate(
                *val_cnd_cxt_targets,
                batch_size=batch_size, report_every=report_every)
        self._report()
    
    def _train_batch(self, cnd, cxt, targets, temp=1.0, lr=0.001):
        _, loss = self._sess.run(
            [self._train, self._loss],
            feed_dict={
                self._inp_cnd: cnd,
                self._inp_cxt: cxt,
                self._targets: targets,
                self._temperature: temp,
                self._lr: lr})
        self._examples.append(self._examples[-1] + cnd.shape[0])
        self._train_losses.append(loss)
        self._temperatures.append(temp)
        self._learn_rates.append(lr)
        self._batches += 1
    
    def _train_epoch(
            self, cnd, cxt, targets,
            batch_size=64, temp_halflife=1000 * 1000, lr=0.001, report_every=100):
        ixs = np.arange(cnd.shape[0])
        np.random.shuffle(ixs)
        batch_slice_starts = np.arange(0, cnd.shape[0], batch_size)
        for i, slice_start in enumerate(batch_slice_starts):
            batch_ixs = ixs[slice_start:slice_start + batch_size]
            xmpl = float(self._examples[-1])
            temp = 0.05 + (0.95 * (0.5 ** (xmpl / temp_halflife)))
            self._lrs.append(lr(self._examples[-1]) if callable(lr) else lr)
            self._train_batch(
                cnd[batch_ixs],
                cxt[batch_ixs],
                targets[batch_ixs],
                temp=temp,
                lr=self._lrs[-1])
            if report_every is not None and (i + 1) % report_every == 0:
                self._report(progress=float(slice_start) / cnd.shape[0])
            if not np.isfinite(self._train_losses[-1]):
                print "Encountered infinite loss!"
                return
        self._epochs += 1

    def _report(self, progress=0.0):
        if self.verbose == 0:
            return
        clear_output(wait=True)
        reports = (
            ("Epoch", self._epochs),
            ("Batch", self._batches),
            ("Epoch Progress", "{:.1f}%".format(progress * 100)),
            ("Learning Rate", self._lrs[-1]),
            ("Examples Seen", self._examples[-1]),
            ("Temperature", self._temperatures[-1]),
            ("Train Loss (last batch)", self._train_losses[-1]),
            ("Train Loss (avg last 100 batches)",
             np.mean(self._train_losses[-100:])),
            ("Last Validation Loss",
             self._valid_losses[-1] if self._valid_losses else None))
        just = max(len(l) for l, _ in reports)
        for label, value in reports:
            print "{}: {}".format(label.rjust(just), value)

    def _validate(self, cnd, cxt, targets, batch_size=64, report_every=100):
        batch_starts = np.arange(0, cnd.shape[0], batch_size)
        losses = []
        for i, start in enumerate(batch_starts):
            end = start + batch_size
            losses.append(self._sess.run([self._loss], feed_dict={
                self._inp_cnd: cnd[start:end],
                self._inp_cxt: cxt[start:end],
                self._targets: targets[start:end],
                self._temperature: 0.05})[0])
            if report_every is not None and (i + 1) % report_every == 0:
                self._validation_report(start, cnd.shape[0])
        self._valid_examples.append(self._examples[-1])
        self._valid_losses.append(np.mean(losses))
            
    def _validation_report(self, so_far, out_of):
        if self.verbose == 0:
            return
        clear_output(wait=True)
        print "Validating, {}/{}".format(so_far, out_of)
    
    def _build(self):
        with tf.variable_scope(self._name):
            self._build_inputs()
            self._build_context_encoder()
            self._build_selector()
            self._build_loss()
    
    def _build_inputs(self):
        with tf.variable_scope("Inputs"):
            self._inp_cnd = tf.placeholder(
                tf.float32, shape=[None, self._n_cnd, self._cnd_ch],
                name="CandidatesInput")
            self._inp_cxt = tf.placeholder(
                tf.float32, shape=[None, self._cxt_ch],
                name="ContextInput")
            self._temperature = tf.placeholder(
                tf.float32, shape=[],
                name="Temperature")
    
    def _build_context_encoder(self):
        with tf.variable_scope("InitialContextEncoder"):
            next_input = self._inp_cxt
            for ch in self._cxt_enc_ch_pattern:
                next_input = tf.contrib.layers.fully_connected(
                    next_input, ch, leaky_relu())
        self._encoded_initial_cxt = tf.expand_dims(
            next_input, axis=1, name="EncodedContext")
    
    def _selector_block(self):
        with tf.variable_scope("SelectorBlock"):
            # Concatenate a copy of the context vector to each candidate
            cxt = self._cxts[-1]
            cxt_tiled = tf.tile(cxt, [1, self._n_cnd, 1])
            sel_hidden = tf.concat(2, [self._inp_cnd, cxt_tiled])
            
            # Apply a non-linear mapping to each candidate to get a fitness score
            for conv in self._sel_convs:
                sel_hidden = conv(sel_hidden)
            
            # Select the candidate with the highest fitness
            eligible = self._eligibilities[-1]
            raw_fitness = eligible * min_max_scale(tf.squeeze(sel_hidden, axis=2), 1.0)
            fitness = self._softmax(raw_fitness, self._temperature)
            selected_ix = tf.argmax(fitness, axis=1)
            self._selection_ixs.append(selected_ix)

            # And get the vector representation of that candidate.
            # To keep it differentiable, instead of using a binary mask, we use
            # a softmax with a temperature parameter that is gradually annealed
            # during training.
            soft_mask = tf.tile(
                tf.expand_dims(fitness, -1),
                [1, 1, self._cnd_ch])
            selected_repr = tf.reduce_mean(
                soft_mask * self._inp_cnd, axis=1, keep_dims=True)

            self._basket_items.append(selected_repr)
            
            # Then mark that candidate as ineligible for future selection
            ineligible = tf.one_hot(selected_ix, self._n_cnd)
            self._eligibilities.append(eligible - ineligible)
            
            # Create the new context vector
            cxt_hidden = tf.concat(2, [selected_repr, cxt])
            for conv in self._cxt_convs:
                cxt_hidden = conv(cxt_hidden)
            self._cxts.append(cxt + cxt_hidden)
    
    def _build_selector(self):
        with tf.variable_scope("Selector"):
            self._basket_items = []
            self._selection_ixs = []
            self._cxts = [self._encoded_initial_cxt]
            self._eligibilities = [
                tf.ones_like(tf.reduce_max(self._inp_cnd, axis=2), name="Eligible")]
            self._sel_convs = reuseable_convs(
                self._sel_ch_pattern, "Selector", final=None)
            self._cxt_convs = reuseable_convs(
                self._cxt_ch_pattern, "Context", final=tf.nn.tanh)
            for _ in np.arange(self._n_choices):
                self._selector_block()
            self._basket = tf.concat(1, self._basket_items)
            self._basket_labels = tf.concat(
                1, [tf.expand_dims(x, axis=1) for x in self._selection_ixs])
    
    def _build_loss(self):
        with tf.variable_scope("Loss"):
            self._targets = tf.placeholder(
                tf.float32, shape=[None, self._n_choices, self._cnd_ch],
                name="Targets")
            best_cos_similarities = tf.reduce_max(
                tf.matmul(
                    tf.nn.l2_normalize(self._targets, 2),
                    tf.nn.l2_normalize(self._basket, 2),
                    transpose_b=True),
                axis=2)
            self._loss = tf.reduce_mean(1 - best_cos_similarities)            
            self._lr = tf.placeholder(tf.float32, [])
            self._train = tf.train.AdamOptimizer(
                learning_rate=self._lr).minimize(self._loss)